# Analysis of Union College math major student data

## Part 2 - Data Cleaning

* Renamed columns to be consistent, brief and pythonic
<br><br>
* Took steps to de-identify data
  + removed student names
  + reassigned ids
<br><br>
* Dropped information that was redundant or unwanted
  + removed rows that were exact duplicates of a previous row
  + dropped MTH-000 courses
<br><br>
* Added columns for derived information
  + Date in datetime format based on term
  + Year and term in separate columns
  + Grade point as a float from letter grade
  + GPA by student from grade points
<br><br>
* Saved cleaned data to new excel file

### Set up the notebook environment

In [1]:
# Imports
import numpy as np
import pandas as pd
import union_constants as uc

In [2]:
# File names
raw_data_fn = 'data/union_students_818.xlsx'
cleaned_data_fn = 'data/union_students_818_cleaned.xlsx'

### Load and clean the data

In [3]:
# Load student info data from excel file
info = pd.read_excel(raw_data_fn, sheet_name='student info')

# Column name style preferences
info.columns = map(str.lower, info.columns)
info.rename({'@id':'id',
             'class - not grad year necessarily':'class_year',
             'total giving':'giving',
             'greek affilation':'greek',
             'student interest 1':'interest_1',
             'student interest 2':'interest_2',
             'student interest 3':'interest_3',
             'student interest 4':'interest_4',
             'student interest5':'interest_5',
             'student interest6':'interest_6',
             'home addr1':'home_address',
             'home city':'home_city',
             'home st':'home_state',
             'left without graduating?':'left_wo_grad'},
            axis=1, inplace=True)

# De-identify ids
#   Reassign student ids to unknown student-id pairings.
unique_ids = info['id'].unique()
np.random.shuffle(unique_ids)
new_ids = {unique_id:idx + 1 for idx, unique_id in enumerate(unique_ids)}
info['id'] = info.apply(lambda srs: new_ids[srs['id']], axis=1)

# Drop names
info.drop(['first', 'last'], axis=1, inplace=True)

# Sort
info.sort_values('id', inplace=True)

In [4]:
# Load grades data from excel file
grades = pd.read_excel(raw_data_fn, sheet_name='psa.grad.grades')

# Column name style preferences
grades.columns = map(str.lower, grades.columns)
grades.rename({'acad major':'major'}, axis=1, inplace=True)

# Drop duplicate rows
grades.drop_duplicates(inplace=True)

# Display and drop MTH-000 students
print('Dropped MTH-000:')
display(grades.loc[grades.course == 'MTH-000'].drop('name', axis=1))  
grades.drop(grades.loc[grades.course == 'MTH-000'].index, inplace=True)

# Add year column and reassign term to season only
grades['year'] = grades['term'].map(lambda srs: int('20' + srs.split('/')[0]))
grades['term'] = grades['term'].map(lambda srs: srs.split('/')[1])

# Derive grade point and date columns
grades['point'] = grades['grade'].map(lambda grade: uc.grade_to_point[grade])
grades['date'] = (grades.apply(lambda srs: pd.to_datetime('{0}/01/{1}'
                                .format(uc.term_to_month[srs.term], srs.year)),
                         axis=1))

# Use de-identifies ids
grades['id'] = grades.apply(lambda srs: new_ids[srs['id']], axis=1)

# Store columns of interest in desired order
grades = grades.loc[:, ['id', 'major', 'course', 'faculty', 'term', 'year', 
                        'date', 'grade', 'point']]

# Sort by new ids
grades.sort_values(['id', 'date'], inplace=True)

Dropped MTH-000:


,id,major,grade,course,faculty,term
97,2453258,MT,NC,MTH-000,"Staff, M.",17/SU
964,2429685,BMTCS,NC,MTH-000,"Staff, M.",16/SU
1255,2461296,LEEMT,NC,MTH-000,"Staff, M.",17/SU


In [5]:
# Calculate GPA by student and add to student info
info = info.merge(grades.groupby('id')
                        .point.agg('mean')
                        .to_frame(name='gpa').reset_index(), 
                  on='id')

In [6]:
# Add major to student info and drop from grades
info = info.merge(grades.groupby('id')
                        .apply(lambda group: group.major.unique()[0])
                        .to_frame(name='major').reset_index(), 
                  on='id')

grades.drop('major', axis=1, inplace=True)

### Display truncated versions of resulting tables

In [7]:
# Display sample of data table
#   Note: The row index, the value in the left-most column, is simply a unique 
#         row identifier. A style function to hide the index from display 
#         exists but is not compatible (as of v0.23.4) with large dataframe 
#         representation (large repr) styling
print('Cleaned student info data table (truncated):')
with pd.option_context('max_row', 5):
    display(info)                              # Large repr, but shows index
    #display(info.head(3).style.hide_index())  # Hide index, but no large repr

Cleaned student info data table (truncated):


,id,class_year,giving,greek,interest_1,interest_2,interest_3,interest_4,interest_5,interest_6,employer,title,home_address,home_city,home_state,left_wo_grad,gpa,major
0,1,2016,0.00,NaN,Fencing Club,Iris Theme House,Math Club,Messa House,NaN,NaN,Randstad North America,Contractor,11 Mary Mount Rd,Auburndale,MA,NaN,3.181818,MT
1,2,2016,0.00,NaN,Women's Rugby Club,Breazzano House,Steinmetz Symposium,NaN,NaN,NaN,Nmg Evaluation,Intern,587 Riverside Dr Apt 6F,New York,NY,NaN,3.500000,BMTPY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,109,2015,0.00,NaN,Steinmetz Symposium,Term Abroad-Mexican-American Border,Minerva House Council,Sorum House,Environmental Club,MEDLIFE,Blackrock Inv. Management,Analyst,Cui Guihua No.36 Chunhe St,Dalian Laioning 116001,NaN,NaN,2.470000,DMTEC
109,110,2016,10.08,NaN,Anime Club,Card and Board Game Club,Chess Club,Breazzano House,NaN,NaN,FDM Group,PMO Trainee,162 Maynard Dr,Amherst,NY,NaN,2.690909,MT


In [8]:
# Display sample table
#   For details on index hiding, see note in info table
print('Cleaned student grades sample data table (truncated):')
with pd.option_context('max_row', 5):
    display(grades)                              # Large repr, but shows index
    #display(data.head(3).style.hide_index())    # Hide index, but no large repr

Cleaned student grades sample data table (truncated):


,id,course,faculty,term,year,date,grade,point
379,1,MTH-113,"Friedman, P.",FA,2012,2012-09-01,A,4.0
381,1,MTH-115,"Zimmermann, K.",WI,2013,2013-01-01,A,4.0
...,...,...,...,...,...,...,...,...
959,110,MTH-497,"Lesh, K.",WI,2016,2016-01-01,B+,3.3
957,110,MTH-238,"Wang, J.",SP,2016,2016-04-01,B+,3.3


### Save student info and grades to excel

In [9]:
# Save tables to Excel
writer = pd.ExcelWriter(cleaned_data_fn, datetime_format='mm/dd/yyyy')
grades.to_excel(writer, sheet_name='grades', index=False)
info.to_excel(writer, sheet_name='info', index=False)
writer.save()